Code taken from https://www.kaggle.com/dromosys/radek-fast-ai-whale-full

In [3]:
%matplotlib inline
import matplotlib.pyplot as plt
from fastai.vision import *
from fastai.metrics import accuracy
from fastai.basic_data import *
from skimage.util import montage
import pandas as pd
from torch import optim
import re

In [6]:
from utils import *

In [8]:
import fastai
from fastprogress import force_console_behavior
import fastprogress

In [9]:
from fastai import *
from fastai.vision import *

In [10]:
path = Path('../data/')
path_test = Path('../data/test')
path_train = Path('../data/train')

In [11]:
df = pd.read_csv(path/'train.csv')#.sample(frac=0.05)
df.head()
val_fns = {'69823499d.jpg'}

In [12]:
df.head()

,Image,Id
0,0000e88ab.jpg,w_f48451c
1,0001f9222.jpg,w_c3d896a
2,00029d126.jpg,w_20df2c5
3,00050a15a.jpg,new_whale
4,0005c1ef8.jpg,new_whale


In [21]:
for row in df.iterrows():
    row[1].Image: row[1].Id 
    print(row[1])
    break

Image    0000e88ab.jpg
Id           w_f48451c
Name: 0, dtype: object


In [22]:
fn2label = {row[1].Image: row[1].Id for row in df.iterrows()}
path2fn = lambda path: re.search('\w*\.jpg$', path).group(0)

In [27]:
name = f'res50-full-train'

In [28]:
SZ = 224
BS = 64
NUM_WORKERS = 16
SEED= 12

In [31]:
data = (
    ImageItemList
        .from_df(df[df.Id != 'new_whale'], path_train, cols=['Image'])
        .split_by_valid_func(lambda path: path2fn(path) in val_fns)
        .label_from_func(lambda path: fn2label[path2fn(path)])
        .add_test(ImageItemList.from_folder(path_test))
        .transform(get_transforms(do_flip=False), size=SZ, resize_method=ResizeMethod.SQUISH)
        .databunch(bs=BS, num_workers=NUM_WORKERS, path=path)
        .normalize(imagenet_stats)
)

In [34]:
MODEL_PATH = "../model/"

In [35]:
%%time

learn = create_cnn(data, models.resnet50, lin_ftrs=[2048], model_dir=MODEL_PATH)
learn.clip_grad();

CPU times: user 3.54 s, sys: 1.1 s, total: 4.64 s
Wall time: 3.89 s


In [36]:
learn.fit_one_cycle(14, 1e-2)
learn.save(f'{name}-stage-1', return_path=True)

epoch,train_loss,valid_loss
1,7.515553,2.653122
2,6.730344,0.093223
3,6.112923,0.103446
4,5.234715,1.646007
5,4.295609,0.024051
6,3.472239,2.460688
7,2.714802,0.000292
8,1.926583,0.000237
9,1.244927,0.002305
10,0.754129,0.036334


In [44]:
learn.unfreeze()

max_lr = 1e-3
lrs = [max_lr/100, max_lr/10, max_lr]

learn.fit_one_cycle(24, lrs)
learn.save(f'{name}-stage-2')

epoch,train_loss,valid_loss
1,0.107084,0.000062
2,0.123389,0.000059
3,0.164280,0.000010
4,0.229229,0.000031
5,0.295162,0.000226
6,0.319031,0.000034
7,0.334586,0.000000
8,0.284205,0.000187
9,0.297835,0.000000
10,0.232849,0.004639


In [48]:
interp = learn.interpret()

In [60]:
SZ = 224 * 2
BS = 64 // 4
NUM_WORKERS = 16
SEED = 12

In [61]:
data = (
    ImageItemList
        .from_df(df[df.Id != 'new_whale'], path_train, cols=['Image'])
        .split_by_valid_func(lambda path: path2fn(path) in val_fns)
        .label_from_func(lambda path: fn2label[path2fn(path)])
        .add_test(ImageItemList.from_folder(path_test))
        .transform(get_transforms(do_flip=False), size=SZ, resize_method=ResizeMethod.SQUISH)
        .databunch(bs=BS, num_workers=NUM_WORKERS, path=path)
        .normalize(imagenet_stats)
)

In [62]:
%%time
learn = create_cnn(data, models.resnet50, lin_ftrs=[2048], model_dir=MODEL_PATH)
learn.clip_grad();
learn.load(f'{name}-stage-2')
learn.freeze_to(-1)

CPU times: user 1.8 s, sys: 407 ms, total: 2.21 s
Wall time: 1.42 s


In [63]:
learn.fit_one_cycle(12, 1e-2 / 4)
learn.save(f'{name}-stage-3')

epoch,train_loss,valid_loss
1,1.189155,0.000000
2,0.864935,0.000000
3,1.421264,0.000000
4,1.755095,0.000000
5,1.936461,0.000000
6,1.711179,0.000004
7,1.599403,0.000000
8,1.344509,0.000005
9,1.109901,0.000000
10,0.943459,0.000000


In [64]:
learn.unfreeze()

max_lr = 1e-3 / 4
lrs = [max_lr/100, max_lr/10, max_lr]

learn.fit_one_cycle(22, lrs)
learn.save(f'{name}-stage-4')

epoch,train_loss,valid_loss
1,0.557895,0.000000
2,0.529344,0.000000
3,0.616036,0.000000
4,0.661306,0.000000
5,0.639274,0.000000
6,0.808095,0.000000
7,0.716195,0.000000
8,0.788330,0.000000
9,0.798525,0.000000
10,0.760456,0.000000


In [65]:
df = pd.read_csv(path/'oversampled_train_and_val.csv')

In [66]:
data = (
    ImageItemList
        .from_df(df, path_train, cols=['Image'])
        .split_by_valid_func(lambda path: path2fn(path) in val_fns)
        .label_from_func(lambda path: fn2label[path2fn(path)])
        .add_test(ImageItemList.from_folder(path_test))
        .transform(get_transforms(do_flip=False), size=SZ, resize_method=ResizeMethod.SQUISH)
        .databunch(bs=BS, num_workers=NUM_WORKERS, path=path)
        .normalize(imagenet_stats)
)

In [67]:
%%time
learn = create_cnn(data, models.resnet50, lin_ftrs=[2048], model_dir=MODEL_PATH)
learn.clip_grad();
learn.load(f'{name}-stage-4')
learn.freeze_to(-1)

CPU times: user 1.83 s, sys: 380 ms, total: 2.21 s
Wall time: 1.42 s


In [68]:
learn.fit_one_cycle(2, 1e-2 / 4)
learn.save(f'{name}-stage-5')

epoch,train_loss,valid_loss
1,2.002180,0.000028
2,0.769943,0.000001


In [69]:
learn.unfreeze()

max_lr = 1e-3 / 4
lrs = [max_lr/100, max_lr/10, max_lr]

learn.fit_one_cycle(3, lrs)
learn.save(f'{name}-stage-6')

epoch,train_loss,valid_loss
1,0.873727,0.000010
2,1.015697,0.000318
3,0.825094,0.000008


In [70]:
preds, _ = learn.get_preds(DatasetType.Test)

In [71]:
preds = torch.cat((preds, torch.ones_like(preds[:, :1])), 1)

In [84]:
preds[:, 5004] = 0.06

In [80]:
classes = learn.data.classes + ['new_whale']

In [81]:
create_submission(preds, learn.data, name, classes)

In [82]:
pd.read_csv(f'subs/{name}.csv.gz').head()

,Image,Id
0,8a4b5372a.jpg,w_8affab2 new_whale w_94af7fd w_bf41c21 w_2972f31
1,6894a2017.jpg,w_9c506f6 new_whale w_5a3e0de w_bb4f52e w_d494351
2,efd1ae9d2.jpg,w_2f1488c new_whale w_33696ac w_b17471a w_02c7e9d
3,6c48e173b.jpg,new_whale w_8cb5e04 w_cc1784d w_e026c82 w_5fcf350
4,7af17248d.jpg,w_69cce86 new_whale w_759b919 w_6f22173 w_1a96d7b


In [83]:
pd.read_csv(f'subs/{name}.csv.gz').Id.str.split().apply(lambda x: x[0] == 'new_whale').mean()

0.5516331658291457

In [78]:
!kaggle competitions submit -c humpback-whale-identification -f subs/{name}.csv.gz -m "{name}"

100%|████████████████████████████████████████| 184k/184k [00:03<00:00, 61.0kB/s]
Successfully submitted to Humpback Whale Identification